In [2]:
!pip install plotly scikit-learn pandas numpy matplotlib seaborn

In [12]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.figure_factory as ff
from sklearn.cluster import KMeans, DBSCAN, AgglomerativeClustering
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.decomposition import PCA
from sklearn.metrics import silhouette_score, calinski_harabasz_score
from sklearn.ensemble import RandomForestClassifier, IsolationForest
from sklearn.model_selection import train_test_split
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

class RetailIntelligencePlatform:
    def __init__(self):
        self.customer_data = None
        self.transaction_data = None
        self.rfm_data = None
        self.segments = None
        self.models = {}
        self.insights = {}

    def generate_synthetic_data(self, n_customers=1000, n_transactions=5000):
        print("🏪 Generating synthetic retail transaction data...")
        np.random.seed(42)
        customer_ids = range(1, n_customers + 1)
        age_groups = np.random.choice(['18-25', '26-35', '36-45', '46-55', '56-65', '65+'],
                                     n_customers, p=[0.15, 0.25, 0.20, 0.20, 0.15, 0.05])
        genders = np.random.choice(['Male', 'Female', 'Other'],
                                  n_customers, p=[0.45, 0.52, 0.03])
        cities = np.random.choice(['New York', 'Los Angeles', 'Chicago', 'Houston', 'Phoenix',
                                  'Philadelphia', 'San Antonio', 'San Diego', 'Dallas', 'Austin'],
                                 n_customers)
        start_date = datetime.now() - timedelta(days=730)
        acquisition_dates = [start_date + timedelta(days=np.random.randint(0, 730))
                           for _ in range(n_customers)]
        self.customer_data = pd.DataFrame({
            'customer_id': customer_ids,
            'age_group': age_groups,
            'gender': genders,
            'city': cities,
            'acquisition_date': acquisition_dates,
            'preferred_channel': np.random.choice(['Online', 'In-Store', 'Mobile App'],
                                                 n_customers, p=[0.45, 0.35, 0.20])
        })
        transactions = []
        product_categories = ['Electronics', 'Clothing', 'Home & Garden', 'Sports', 'Books',
                            'Beauty', 'Groceries', 'Automotive', 'Toys', 'Health']
        for i in range(n_transactions):
            customer_weights = np.random.exponential(1, n_customers)
            customer_weights = customer_weights / customer_weights.sum()
            customer_id = np.random.choice(customer_ids, p=customer_weights)
            customer_info = self.customer_data[self.customer_data['customer_id'] == customer_id].iloc[0]
            acq_date = customer_info['acquisition_date']
            days_since_acq = (datetime.now() - acq_date).days
            if days_since_acq <= 0:
                days_since_acq =
            transaction_date = acq_date + timedelta(days=np.random.randint(0, days_since_acq))
            if customer_id <= n_customers * 0.1:
                base_amount = np.random.normal(200, 50)
                quantity = np.random.randint(3, 8)
                frequency_modifier = 3
            elif customer_id <= n_customers * 0.3:
                base_amount = np.random.normal(100, 30)
                quantity = np.random.randint(2, 5)
                frequency_modifier = 2
            elif customer_id <= n_customers * 0.7:
                base_amount = np.random.normal(60, 20)
                quantity = np.random.randint(1, 4)
                frequency_modifier = 1
            else:
                base_amount = np.random.normal(35, 15)
                quantity = np.random.randint(1, 3)
                frequency_modifier = 0.5
            month = transaction_date.month
            if month in [11, 12]:
                base_amount *= 1.5
                quantity = int(quantity * 1.3)
            elif month in [6, 7, 8]:
                base_amount *= 0.8
            amount = max(10, base_amount)
            quantity = max(1, quantity)
            if customer_info['age_group'] in ['18-25', '26-35']:
                category_probs = [0.2, 0.25, 0.1, 0.15, 0.05, 0.1, 0.05, 0.05, 0.05, 0.0]
            elif customer_info['age_group'] in ['36-45', '46-55']:
                category_probs = [0.15, 0.15, 0.2, 0.1, 0.1, 0.1, 0.1, 0.05, 0.05, 0.0]
            else:
                category_probs = [0.1, 0.1, 0.25, 0.05, 0.15, 0.05, 0.15, 0.05, 0.05, 0.05]
            category = np.random.choice(product_categories, p=category_probs)
            transactions.append({
                'transaction_id': f'TXN_{i+1:06d}',
                'customer_id': customer_id,
                'transaction_date': transaction_date,
                'amount': round(amount, 2),
                'quantity': quantity,
                'category': category,
                'channel': customer_info['preferred_channel'],
                'discount_applied': np.random.choice([0, 5, 10, 15, 20], p=[0.4, 0.3, 0.15, 0.1, 0.05])
            })
        self.transaction_data = pd.DataFrame(transactions)
        print(f"✅ Generated {len(self.customer_data)} customers and {len(self.transaction_data)} transactions")
        return self.customer_data, self.transaction_data

    def calculate_rfm_metrics(self):
        print("📊 Calculating advanced RFM metrics...")
        current_date = self.transaction_data['transaction_date'].max()
        rfm = self.transaction_data.groupby('customer_id').agg({
            'transaction_date': lambda x: (current_date - x.max()).days,
            'transaction_id': 'count',
            'amount': ['sum', 'mean', 'std'],
            'quantity': ['sum', 'mean'],
            'discount_applied': 'mean',
            'category': lambda x: len(x.unique())
        }).reset_index()
        rfm.columns = ['customer_id', 'recency', 'frequency', 'monetary_total',
                       'monetary_avg', 'monetary_std', 'quantity_total',
                       'quantity_avg', 'avg_discount', 'category_diversity']
        rfm['monetary_std'] = rfm['monetary_std'].fillna(0)
        rfm['days_since_first_purchase'] = self.transaction_data.groupby('customer_id')['transaction_date'].agg(
            lambda x: (current_date - x.min()).days
        ).values
        rfm['purchase_span'] = rfm['days_since_first_purchase'] - rfm['recency']
        rfm['purchase_frequency_rate'] = rfm['frequency'] / (rfm['purchase_span'] + 1)
        rfm['estimated_clv'] = (rfm['monetary_avg'] * rfm['purchase_frequency_rate'] * 365) / (1 + rfm['recency']/365)
        rfm['consistency_score'] = 1 / (1 + rfm['monetary_std'] / (rfm['monetary_avg'] + 1))
        rfm['loyalty_score'] = (rfm['frequency'] * rfm['category_diversity']) / (rfm['recency'] + 1)
        self.rfm_data = rfm
        print(f"✅ Calculated RFM metrics for {len(rfm)} customers")
        return rfm

    def perform_advanced_segmentation(self):
        print("🤖 Performing advanced customer segmentation...")
        features = ['recency', 'frequency', 'monetary_total', 'monetary_avg',
                   'category_diversity', 'estimated_clv', 'loyalty_score', 'consistency_score']
        X = self.rfm_data[features].copy()
        X = X.fillna(X.mean())
        scaler = RobustScaler()
        X_scaled = scaler.fit_transform(X)
        silhouette_scores = []
        calinski_scores = []
        inertias = []
        k_range = range(3, 11)
        for k in k_range:
            kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
            cluster_labels = kmeans.fit_predict(X_scaled)
            silhouette_scores.append(silhouette_score(X_scaled, cluster_labels))
            calinski_scores.append(calinski_harabasz_score(X_scaled, cluster_labels))
            inertias.append(kmeans.inertia_)
        optimal_k = k_range[np.argmax(silhouette_scores)]
        print(f"🎯 Optimal number of clusters: {optimal_k}")
        clustering_results = {}
        kmeans = KMeans(n_clusters=optimal_k, random_state=42, n_init=10)
        clustering_results['kmeans'] = kmeans.fit_predict(X_scaled)
        hierarchical = AgglomerativeClustering(n_clusters=optimal_k)
        clustering_results['hierarchical'] = hierarchical.fit_predict(X_scaled)
        dbscan = DBSCAN(eps=0.5, min_samples=5)
        clustering_results['dbscan'] = dbscan.fit_predict(X_scaled)
        self.rfm_data['segment'] = clustering_results['kmeans']
        self.models['scaler'] = scaler
        self.models['kmeans'] = kmeans
        self.models['clustering_results'] = clustering_results
        self.models['feature_names'] = features
        pca = PCA(n_components=2)
        X_pca = pca.fit_transform(X_scaled)
        self.rfm_data['pca_1'] = X_pca[:, 0]
        self.rfm_data['pca_2'] = X_pca[:, 1]
        self.models['pca'] = pca
        print(f"✅ Segmentation complete with {len(np.unique(clustering_results['kmeans']))} segments")
        return clustering_results, optimal_k

    def analyze_segments(self):
        print("🔍 Analyzing customer segments...")
        segment_analysis = self.rfm_data.groupby('segment').agg({
            'recency': ['mean', 'median'],
            'frequency': ['mean', 'median'],
            'monetary_total': ['mean', 'median', 'sum'],
            'monetary_avg': ['mean', 'median'],
            'estimated_clv': ['mean', 'median', 'sum'],
            'loyalty_score': ['mean', 'median'],
            'consistency_score': ['mean', 'median'],
            'category_diversity': ['mean', 'median'],
            'customer_id': 'count'
        }).round(2)
        segment_analysis.columns = [f"{col[0]}_{col[1]}" for col in segment_analysis.columns]
        segment_analysis = segment_analysis.reset_index()
        segment_names = {}
        segment_descriptions = {}
        marketing_strategies = {}
        for idx, row in segment_analysis.iterrows():
            segment = row['segment']
            if row['monetary_total_mean'] > segment_analysis['monetary_total_mean'].quantile(0.8):
                if row['frequency_mean'] > segment_analysis['frequency_mean'].quantile(0.7):
                    name = "💎 VIP Champions"
                    description = "High-value, frequent buyers - our most valuable customers"
                    strategy = "Exclusive perks, early access, personal service"
                else:
                    name = "💰 Big Spenders"
                    description = "High-value but infrequent buyers"
                    strategy = "Luxury products, premium experiences, re-engagement campaigns"
            elif row['frequency_mean'] > segment_analysis['frequency_mean'].quantile(0.8):
                if row['recency_mean'] < segment_analysis['recency_mean'].quantile(0.3):
                    name = "🔥 Loyal Enthusiasts"
                    description = "Frequent, recent buyers with moderate spend"
                    strategy = "Loyalty rewards, cross-selling, referral programs"
                else:
                    name = "⚠ At-Risk Loyalists"
                    description = "Previously frequent buyers showing decline"
                    strategy = "Win-back campaigns, special offers, satisfaction surveys"
            elif row['recency_mean'] < segment_analysis['recency_mean'].quantile(0.4):
                name = "🌱 New Promising"
                description = "Recent customers with good potential"
                strategy = "Onboarding programs, product education, engagement campaigns"
            elif row['recency_mean'] > segment_analysis['recency_mean'].quantile(0.7):
                if row['monetary_total_mean'] > segment_analysis['monetary_total_mean'].median():
                    name = "💔 Lost High-Value"
                    description = "High-value customers who haven't purchased recently"
                    strategy = "Aggressive win-back, surveys, special incentives"
                else:
                    name = "😴 Hibernating"
                    description = "Low engagement, dormant customers"
                    strategy = "Low-cost reactivation, email campaigns, surveys"
            else:
                name = "🎯 Core Customers"
                description = "Balanced customers - steady, reliable base"
                strategy = "Retention focus, upselling, consistent engagement"
            segment_names[segment] = name
            segment_descriptions[segment] = description
            marketing_strategies[segment] = strategy
        segment_analysis['segment_name'] = segment_analysis['segment'].map(segment_names)
        segment_analysis['description'] = segment_analysis['segment'].map(segment_descriptions)
        segment_analysis['marketing_strategy'] = segment_analysis['segment'].map(marketing_strategies)
        self.rfm_data['segment_name'] = self.rfm_data['segment'].map(segment_names)
        self.segments = {
            'analysis': segment_analysis,
            'names': segment_names,
            'descriptions': segment_descriptions,
            'strategies': marketing_strategies
        }
        print(f"✅ Analyzed {len(segment_names)} distinct customer segments")
        return segment_analysis

    def detect_anomalies(self):
        print("🔍 Detecting customer behavior anomalies...")
        features = self.models['feature_names']
        X = self.rfm_data[features].fillna(self.rfm_data[features].mean())
        X_scaled = self.models['scaler'].transform(X)
        iso_forest = IsolationForest(contamination=0.1, random_state=42)
        anomaly_scores = iso_forest.fit_predict(X_scaled)
        anomaly_scores_numeric = iso_forest.decision_function(X_scaled)
        self.rfm_data['is_anomaly'] = anomaly_scores == -1
        self.rfm_data['anomaly_score'] = anomaly_scores_numeric
        anomalies = self.rfm_data[self.rfm_data['is_anomaly']]
        print(f"🚨 Detected {len(anomalies)} anomalous customers ({len(anomalies)/len(self.rfm_data)*100:.1f}%)")
        return anomalies

    def create_comprehensive_dashboard(self):
        print("📊 Creating comprehensive analytics dashboard...")
        fig = make_subplots(
            rows=4, cols=2,
            subplot_titles=(
                'Customer Segment Distribution', 'RFM 3D Scatter Plot',
                'Segment Performance Metrics', 'Customer Lifetime Value Distribution',
                'Segment Recency vs Frequency', 'Monthly Revenue by Segment',
                'Category Preferences by Segment', 'Anomaly Detection Visualization'
            ),
            specs=[
                [{"type": "pie"}, {"type": "scatter3d"}],
                [{"type": "bar"}, {"type": "histogram"}],
                [{"type": "scatter"}, {"type": "bar"}],
                [{"type": "bar"}, {"type": "scatter"}]
            ],
            vertical_spacing=0.08
        )
        segment_counts = self.rfm_data['segment_name'].value_counts()
        fig.add_trace(go.Pie(
            labels=segment_counts.index,
            values=segment_counts.values,
            name="Segments",
            textinfo='label+percent',
            textposition='inside'
        ), row=1, col=1)
        colors = px.colors.qualitative.Set3[:len(self.rfm_data['segment'].unique())]
        for i, segment in enumerate(self.rfm_data['segment'].unique()):
            segment_data = self.rfm_data[self.rfm_data['segment'] == segment]
            segment_name = segment_data['segment_name'].iloc[0]
            fig.add_trace(go.Scatter3d(
                x=segment_data['recency'],
                y=segment_data['frequency'],
                z=segment_data['monetary_total'],
                mode='markers',
                marker=dict(
                    size=4,
                    color=colors[i % len(colors)],
                    opacity=0.7
                ),
                name=segment_name,
                text=segment_data['customer_id'],
                showlegend=False
            ), row=1, col=2)
        metrics = ['monetary_total_mean', 'frequency_mean', 'estimated_clv_mean']
        segment_analysis = self.segments['analysis']
        for metric in metrics:
            fig.add_trace(go.Bar(
                x=segment_analysis['segment_name'],
                y=segment_analysis[metric],
                name=metric.replace('mean', '').replace('_', ' ').title(),
                showlegend=True if metric == metrics[0] else False
            ), row=2, col=1)
        fig.add_trace(go.Histogram(
            x=self.rfm_data['estimated_clv'],
            nbinsx=30,
            name="CLV Distribution",
            opacity=0.7
        ), row=2, col=2)
        for i, segment in enumerate(self.rfm_data['segment'].unique()):
            segment_data = self.rfm_data[self.rfm_data['segment'] == segment]
            segment_name = segment_data['segment_name'].iloc[0]
            fig.add_trace(go.Scatter(
                x=segment_data['recency'],
                y=segment_data['frequency'],
                mode='markers',
                marker=dict(
                    size=segment_data['monetary_total'] / 50,
                    color=colors[i % len(colors)],
                    opacity=0.6
                ),
                name=segment_name,
                text=segment_data['estimated_clv'],
                showlegend=False
            ), row=3, col=1)
        months = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun']
        for segment_name in segment_counts.index[:3]:
            revenue = np.random.normal(50000, 10000, 6)
            fig.add_trace(go.Bar(
                x=months,
                y=revenue,
                name=segment_name,
                showlegend=False
            ), row=3, col=2)
        category_data = self.transaction_data.merge(
            self.rfm_data[['customer_id', 'segment_name']],
            on='customer_id'
        )
        category_segment = category_data.groupby(['segment_name', 'category'])['amount'].sum().reset_index()
        for segment in category_segment['segment_name'].unique()[:3]:
            segment_cat_data = category_segment[category_segment['segment_name'] == segment]
            fig.add_trace(go.Bar(
                x=segment_cat_data['category'],
                y=segment_cat_data['amount'],
                name=segment,
                showlegend=False
            ), row=4, col=1)
        anomalies = self.rfm_data[self.rfm_data['is_anomaly']]
        normal_customers = self.rfm_data[~self.rfm_data['is_anomaly']]
        fig.add_trace(go.Scatter(
            x=normal_customers['pca_1'],
            y=normal_customers['pca_2'],
            mode='markers',
            marker=dict(color='blue', size=4, opacity=0.6),
            name='Normal',
            showlegend=False
        ), row=4, col=2)
        fig.add_trace(go.Scatter(
            x=anomalies['pca_1'],
            y=anomalies['pca_2'],
            mode='markers',
            marker=dict(color='red', size=8, symbol='x'),
            name='Anomalies',
            showlegend=False
        ), row=4, col=2)
        fig.update_layout(
            height=1600,
            title_text="🏪 AI Retail Intelligence Dashboard",
            title_font_size=20,
            showlegend=True
        )
        fig.show()
        self._create_advanced_visualizations()
        return fig

    def _create_advanced_visualizations(self):
        fig_journey = go.Figure()
        segments = self.rfm_data['segment_name'].unique()
        for segment in segments:
            segment_data = self.rfm_data[self.rfm_data['segment_name'] == segment]
            max_freq = max(segment_data['frequency']) if len(segment_data) > 0 else 1
            fig_journey.add_trace(go.Scatter(
                x=segment_data['days_since_first_purchase'],
                y=segment_data['estimated_clv'],
                mode='markers',
                name=segment,
                marker=dict(
                    size=segment_data['frequency'],
                    opacity=0.6,
                    sizemode='diameter',
                    sizeref=2. * max_freq / (40. ** 2),
                    sizemin=4
                ),
                text=[f"Customer: {cid}<br>Frequency: {freq}<br>CLV: ${clv:.2f}"
                      for cid, freq, clv in zip(segment_data['customer_id'],
                                               segment_data['frequency'],
                                               segment_data['estimated_clv'])]
            ))
        fig_journey.update_layout(
            title="Customer Journey: Tenure vs Customer Lifetime Value",
            xaxis_title="Days Since First Purchase",
            yaxis_title="Estimated Customer Lifetime Value ($)",
            height=600
        )
        fig_journey.show()
        segment_metrics = self.segments['analysis'][['segment_name', 'monetary_total_mean','frequency_mean', 'estimated_clv_mean','loyalty_score_mean', 'consistency_score_mean']]
        for col in segment_metrics.columns[1:]:
            col_min = segment_metrics[col].min()
            col_max = segment_metrics[col].max()
            if col_max - col_min > 0:
                segment_metrics[col] = (segment_metrics[col] - col_min) / (col_max - col_min)
            else:
                segment_metrics[col] = 0
        fig_radar = go.Figure()
        categories = ['Monetary Value', 'Frequency', 'Customer Lifetime Value','Loyalty Score', 'Consistency Score']
        for _, row in segment_metrics.iterrows():
            fig_radar.add_trace(go.Scatterpolar(
                r=[row['monetary_total_mean'], row['frequency_mean'], row['estimated_clv_mean'],
                   row['loyalty_score_mean'], row['consistency_score_mean']],
                theta=categories,
                fill='toself',
                name=row['segment_name']
            ))
        fig_radar.update_layout(
            polar=dict(
                radialaxis=dict(
                    visible=True,
                    range=[0, 1]
                )),
            showlegend=True,
            title="Segment Comparison - Radar Chart",
            height=600
        )
        fig_radar.show()
    def generate_business_insights(self):
        print("💡 Generating business insights and recommendations...")
        insights = {
            'total_customers': len(self.customer_data),
            'total_revenue': self.transaction_data['amount'].sum(),
            'avg_order_value': self.transaction_data['amount'].mean(),
            'total_clv': self.rfm_data['estimated_clv'].sum(),
            'segment_distribution': self.rfm_data['segment_name'].value_counts().to_dict(),
            'top_segment_by_value': self.segments['analysis'].loc[
                self.segments['analysis']['monetary_total_sum'].idxmax(), 'segment_name'
            ],
            'most_loyal_segment': self.segments['analysis'].loc[
                self.segments['analysis']['loyalty_score_mean'].idxmax(), 'segment_name'
            ],
            'at_risk_customers': len(self.rfm_data[self.rfm_data['recency'] > 180]),
            'high_value_customers': len(self.rfm_data[self.rfm_data['estimated_clv'] >
                                                    self.rfm_data['estimated_clv'].quantile(0.8)]),
            'category_performance': self.transaction_data.groupby('category')['amount'].sum().to_dict(),
            'channel_performance': self.transaction_data.groupby('channel')['amount'].sum().to_dict()
        }
        self.insights = insights
        return insights

    def generate_comprehensive_report(self):
        insights = self.generate_business_insights()
        print("\n" + "="*80)
        print("🏪 AI RETAIL INTELLIGENCE PLATFORM - COMPREHENSIVE REPORT")
        print("="*80)
        print(f"\n📊 BUSINESS OVERVIEW:")
        print(f"   • Total Customers: {insights['total_customers']:,}")
        print(f"   • Total Revenue: ${insights['total_revenue']:,.2f}")
        print(f"   • Average Order Value: ${insights['avg_order_value']:.2f}")
        print(f"   • Total Customer Lifetime Value: ${insights['total_clv']:,.2f}")
        print(f"   • Average CLV per Customer: ${insights['total_clv']/insights['total_customers']:.2f}")
        print(f"\n🎯 CUSTOMER SEGMENTATION:")
        for segment, count in insights['segment_distribution'].items():
            percentage = (count / insights['total_customers']) * 100
            print(f"   • {segment}: {count:,} customers ({percentage:.1f}%)")
        print(f"\n💎 TOP PERFORMING SEGMENTS:")
        print(f"   • Highest Value Segment: {insights['top_segment_by_value']}")
        print(f"   • Most Loyal Segment: {insights['most_loyal_segment']}")
        print(f"\n🚨 AT-RISK AND HIGH-VALUE CUSTOMERS:")
        print(f"   • At-Risk Customers (inactive > 180 days): {insights['at_risk_customers']:,}")
        print(f"   • High-Value Customers (top 20% CLV): {insights['high_value_customers']:,}")
        print(f"\n🛍️ CATEGORY PERFORMANCE:")
        for category, revenue in sorted(insights['category_performance'].items(), key=lambda x: x[1], reverse=True):
            print(f"   • {category}: ${revenue:,.2f}")
        print(f"\n📱 CHANNEL PERFORMANCE:")
        for channel, revenue in sorted(insights['channel_performance'].items(), key=lambda x: x[1], reverse=True):
            print(f"   • {channel}: ${revenue:,.2f}")
        print(f"\n🔥 STRATEGIC RECOMMENDATIONS:")
        for segment, row in self.segments['analysis'].iterrows():
            print(f"\n   Segment: {row['segment_name']}")
            print(f"   Description: {self.segments['descriptions'][segment]}")
            print(f"   Strategy: {self.segments['strategies'][segment]}")
            print(f"   Key Metrics:")
            print(f"     - Average Recency: {row['recency_mean']:.1f} days")
            print(f"     - Average Frequency: {row['frequency_mean']:.1f} purchases")
            print(f"     - Average Monetary Value: ${row['monetary_total_mean']:,.2f}")
            print(f"     - Average CLV: ${row['estimated_clv_mean']:,.2f}")
        print("\n" + "="*80)
        print("END OF REPORT")
        print("="*80)
        return insights

if __name__ == "__main__":
    rip = RetailIntelligencePlatform()
    customers, transactions = rip.generate_synthetic_data()
    rfm_metrics = rip.calculate_rfm_metrics()
    clustering_results, optimal_k = rip.perform_advanced_segmentation()
    segment_analysis = rip.analyze_segments()
    anomalies = rip.detect_anomalies()
    dashboard = rip.create_comprehensive_dashboard()
    report = rip.generate_comprehensive_report()

    print("\nSample Customer Data:")
    print(customers.head())
    print("\nSample Transaction Data:")
    print(transactions.head())
    print("\nSample RFM Metrics:")
    print(rfm_metrics.head())
    print("\nSegment Analysis Summary:")
    print(segment_analysis[['segment_name', 'customer_id_count', 'monetary_total_mean', 'frequency_mean', 'estimated_clv_mean']])
    print(f"\nNumber of Anomalies Detected: {len(anomalies)}")

🏪 Generating synthetic retail transaction data...
✅ Generated 1000 customers and 5000 transactions
📊 Calculating advanced RFM metrics...
✅ Calculated RFM metrics for 992 customers
🤖 Performing advanced customer segmentation...
🎯 Optimal number of clusters: 3
✅ Segmentation complete with 3 segments
🔍 Analyzing customer segments...
✅ Analyzed 3 distinct customer segments
🔍 Detecting customer behavior anomalies...
🚨 Detected 100 anomalous customers (10.1%)
📊 Creating comprehensive analytics dashboard...


💡 Generating business insights and recommendations...

🏪 AI RETAIL INTELLIGENCE PLATFORM - COMPREHENSIVE REPORT

📊 BUSINESS OVERVIEW:
   • Total Customers: 1,000
   • Total Revenue: $358,553.32
   • Average Order Value: $71.71
   • Total Customer Lifetime Value: $2,208,254.40
   • Average CLV per Customer: $2208.25

🎯 CUSTOMER SEGMENTATION:
   • ⚠ At-Risk Loyalists: 969 customers (96.9%)
   • 🎯 Core Customers: 21 customers (2.1%)
   • 💰 Big Spenders: 2 customers (0.2%)

💎 TOP PERFORMING SEGMENTS:
   • Highest Value Segment: ⚠ At-Risk Loyalists
   • Most Loyal Segment: 💰 Big Spenders

🚨 AT-RISK AND HIGH-VALUE CUSTOMERS:
   • At-Risk Customers (inactive > 180 days): 82
   • High-Value Customers (top 20% CLV): 199

🛍️ CATEGORY PERFORMANCE:
   • Clothing: $67,696.06
   • Electronics: $59,959.58
   • Home & Garden: $58,075.62
   • Sports: $37,159.25
   • Books: $33,726.95
   • Groceries: $32,504.83
   • Beauty: $31,202.24
   • Toys: $18,968.53
   • Automotive: $17,233.37
   • Health: $2,026